<a href="https://colab.research.google.com/github/Dhinesh-Manikandan/HireIntel-AI-powered-Resume-Ranker/blob/main/AI_Powered_Resume_Ranker_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio sentence-transformers pdfplumber python-docx scikit-learn pandas matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 778.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

In [2]:
import gradio as gr
import os
import tempfile
import pandas as pd
import matplotlib.pyplot as plt
import re

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pdfplumber
from docx import Document

model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# Extract text from PDF or DOCX
def extract_text(file_path, ext):
    try:
        if ext == 'pdf':
            with pdfplumber.open(file_path) as pdf:
                return ' '.join(page.extract_text() or '' for page in pdf.pages)
        elif ext == 'docx':
            return '\n'.join(p.text for p in Document(file_path).paragraphs)
    except:
        return ''
    return ''

In [4]:
# Identify JD
def is_probable_jd(text, filename):
    jd_keywords = [
        "responsibilities", "requirements", "qualifications", "job description", "skills",
        "preferred", "desired", "job summary", "position", "job title"
    ]
    resume_keywords = [
        "education", "projects", "certifications", "linkedin", "experience", "internship", "objective"
    ]
    text_lower = text.lower()
    jd_score = sum(text_lower.count(k) for k in jd_keywords)
    resume_score = sum(text_lower.count(k) for k in resume_keywords)
    filename_hint = "jd" in filename.lower() or "description" in filename.lower()
    return jd_score + (20 if filename_hint else 0) > resume_score


In [17]:

# Keyword extractor
def extract_top_keywords(text1, text2, top_n=5):
    try:
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf = vectorizer.fit_transform([text1, text2])
        print(tfidf)
        print("__"*10)
        feature_names = vectorizer.get_feature_names_out()
        print(feature_names)
        print("__"*10)
        diff = tfidf[0] - tfidf[1]
        print(diff)
        print("__"*10)
        abs_diff = abs(diff.toarray()[0])
        print(abs_diff)
        print("__"*10)
        top_indices = abs_diff.argsort()[-top_n:][::-1]
        print(top_indices)
        print("__"*10)
        return [feature_names[i] for i in top_indices]
    except:
        return []

In [34]:

def score_resumes(files):
    try:
        if not files or len(files) < 2:
            raise gr.Error("Please upload at least 1 JD and 1 resume.")

        text_data, jd_files, resume_files = {}, {}, {}

        # Read and extract text directly from file paths
        for file_path in files:
            ext = file_path.split('.')[-1].lower()
            text = extract_text(file_path, ext)
            text_data[os.path.basename(file_path)] = text

        # Classify into JD and Resumes
        for filename, text in text_data.items():
            if is_probable_jd(text, filename):
                jd_files[filename] = text
            else:
                resume_files[filename] = text

        if not jd_files:
            raise gr.Error("No Job Description detected.")
        if not resume_files:
            raise gr.Error("No resumes detected.")

        results = []
        for resume_name, resume_text in resume_files.items():
            resume_embedding = model.encode([resume_text])
            for jd_name, jd_text in jd_files.items():
                jd_embedding = model.encode([jd_text])
                sim = cosine_similarity(resume_embedding, jd_embedding)[0][0]
                top_keywords = extract_top_keywords(jd_text, resume_text)
                results.append((
                    os.path.basename(resume_name),
                    os.path.basename(jd_name),
                    float(f"{sim * 100:.2f}"),
                    ', '.join(top_keywords)
                ))

        df = pd.DataFrame(results, columns=["Resume", "Job Description", "Match %", "Top Keywords"])
        df = df.sort_values(by="Match %", ascending=False).reset_index(drop=True)

        # Plot
        plt.figure(figsize=(10, 4))
        bars = plt.barh(df["Resume"], df["Match %"], color="blue")
        plt.xlabel("Match %")
        plt.title("Resume Match Score")
        plt.gca().invert_yaxis()
        for bar in bars:
            width = bar.get_width()
            plt.text(width + 1, bar.get_y() + bar.get_height() / 2, f'{width:.2f}%', va='center')
        plot_path = os.path.join(tempfile.gettempdir(), "match_plot.png")
        plt.tight_layout()
        plt.savefig(plot_path)
        plt.close()

        # Save CSV
        csv_path = os.path.join(tempfile.gettempdir(), "resume_ranking_results.csv")
        df.to_csv(csv_path, index=False)

        return df, plot_path, csv_path

    except Exception as e:
        import traceback
        traceback.print_exc()
        raise gr.Error(f"⚠️ Something went wrong:\n\n{e}")

In [35]:
import gradio as gr

with gr.Blocks(title="HireIntel", theme=gr.themes.Base(primary_hue="blue", secondary_hue="yellow")) as demo:
    # Styled and centered title
    gr.HTML("""
        <div style="text-align: center; margin-bottom: 10px;">
            <h1 style="color: #1E90FF; font-size: 36px; font-weight: bold;">🤖 HireIntel</h1>
        </div>
    """)

    gr.Markdown(
        "Upload resumes and job descriptions (`.pdf` or `.docx`) in any order.\n"
        "The system will auto-detect the JD and rank resumes by match score."
    )

    with gr.Row():
        file_input = gr.File(
            file_types=[".pdf", ".docx"],
            file_count="multiple",
            label="📤 Drag or Upload JD & Resumes",
            interactive=True
        )

    with gr.Row():
        calculate_btn = gr.Button("✅ Calculate Score", variant="primary")
        cancel_btn = gr.Button("❌ Cancel", variant="secondary")

    gr.Markdown("---")

    gr.Markdown("## 📋 Resume Match Results")
    output_table = gr.Dataframe(label="")
    output_plot = gr.Image(type="filepath", label="📊 Resume Match Score Chart")

    gr.Markdown("## 📥 Download HR Report")
    output_csv = gr.File(label="Click to Download CSV Report", file_types=[".csv"])

    # Reset button logic
    cancel_btn.click(
        fn=lambda: (None, None, None, None),
        inputs=[],
        outputs=[file_input, output_table, output_plot, output_csv]
    )

    # Calculate button logic
    calculate_btn.click(
        fn=score_resumes,
        inputs=[file_input],
        outputs=[output_table, output_plot, output_csv]
    )

    # Footer credit centered with mild color
    gr.HTML("""
        <div style="text-align: center; margin-top: 30px;">
            <p style="color: #6c757d; font-size: 16px;">🔧 Crafted with ❤️ by <strong>Dhinesh Manikandan</strong></p>
        </div>
    """)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://852d481f1f0838b835.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
